In [1]:
# Import necessary libraries
import requests
from collections import Counter
import pickle
import time

# Configuration
ECOLI_URL = "https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.fna.gz"
TARGET_VOCAB_SIZE = 5000
TARGET_COMPRESSION = 3.2

print("Setup complete!")

Setup complete!


In [2]:
import gzip
import os

def download_ecoli():
    """Download E. coli genome from NCBI"""
    filename = "ecoli_genome.fna.gz"

    if os.path.exists(filename):
        print(f"✓ {filename} already exists")
        return filename

    print("Downloading E. coli genome...")
    response = requests.get(ECOLI_URL)

    with open(filename, 'wb') as f:
        f.write(response.content)

    print(f"✓ Downloaded {len(response.content)} bytes")
    return filename

# Download the file
genome_file = download_ecoli()

✓ Downloaded 1379902 bytes


In [3]:
print(genome_file)

ecoli_genome.fna.gz


In [5]:
def load_fasta(filename):
    """
    Load FASTA file and return DNA sequence as a single string

    Args:
        filename: path to .fna.gz file

    Returns:
        string: concatenated DNA sequence
    """
    sequences = []

    # Open gzip-compressed file
    with gzip.open(filename, 'rt') as f:  # 'rt' = read text mode
        for line in f:
            line = line.strip()  # Remove leading/trailing whitespace

            # Skip empty lines
            if not line:
                continue

            # Skip header lines (start with '>')
            if line.startswith('>'):
                continue

            # This is a sequence line - add it
            sequences.append(line.upper())

    # Concatenate all sequences into one string
    corpus = ''.join(sequences)

    return corpus


# Load the genome
print("Loading E. coli genome...")
corpus = load_fasta(genome_file)

print(f"✓ Loaded {len(corpus):,} base pairs")
print(f"First 100 bases: {corpus[:100]}")
print(f"Character counts: {Counter(corpus)}")

Loading E. coli genome...
✓ Loaded 4,641,652 base pairs
First 100 bases: AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGATAGCAGCTTCTGAACTGGTTACCTGCCGTGAGTAAAT
Character counts: Counter({'C': 1180091, 'G': 1177437, 'A': 1142742, 'T': 1141382})


In [6]:
def initialize_vocab(corpus):
    """
    Create initial vocabulary from unique characters in corpus

    Args:
        corpus: DNA sequence string

    Returns:
        dict mapping characters to IDs
    """
    # Get all unique characters in the corpus
    unique_chars = sorted(set(corpus))

    # Create vocabulary dictionary: character -> ID
    # IDs start from 0 and increment
    vocab = {char: idx for idx, char in enumerate(unique_chars)}

    return vocab


# Initialize
vocab = initialize_vocab(corpus)
print(f"Initial vocabulary: {vocab}")
print(f"Initial vocab size: {len(vocab)}")

Initial vocabulary: {'A': 0, 'C': 1, 'G': 2, 'T': 3}
Initial vocab size: 4


In [8]:
def get_pair_counts(tokens):
    """
    Count frequency of all adjacent token pairs

    Args:
        tokens: list of current tokens (strings)

    Returns:
        Counter object with {(token1, token2): count}
    """
    # Create empty counter to store pair frequencies
    pairs = []

    # Iterate through tokens, creating adjacent pairs
    # Stop at len(tokens)-1 because we need token[i] and token[i+1]
    for i in range(len(tokens) - 1):
        pair = (tokens[i], tokens[i + 1])
        pairs.append(pair)

    # Count how many times each pair appears
    pair_counts = Counter(pairs)

    return pair_counts


def merge_pair(tokens, pair, new_token):
    """
    Replace all occurrences of pair with new_token

    Args:
        tokens: list of current tokens
        pair: tuple (token1, token2) to merge
        new_token: string to replace pair with

    Returns:
        new list of tokens with pairs merged
    """
    new_tokens = []
    i = 0

    # Iterate through tokens
    while i < len(tokens):
        # Check if we found the pair we're looking for
        # Need to check i+1 exists and that current + next match the pair
        if i < len(tokens) - 1 and (tokens[i], tokens[i + 1]) == pair:
            # Found the pair! Replace with merged token
            new_tokens.append(new_token)
            i += 2  # Skip both tokens in the pair
        else:
            # Not the pair we're looking for, keep token as-is
            new_tokens.append(tokens[i])
            i += 1  # Move to next token

    return new_tokens


# Test function
print("Testing BPE functions...")
test_tokens = ['A', 'T', 'C', 'G', 'A', 'T']
print(f"Test tokens: {test_tokens}")

test_counts = get_pair_counts(test_tokens)
print(f"Pair counts: {test_counts}")

test_merged = merge_pair(test_tokens, ('A', 'T'), 'AT')
print(f"After merging ('A', 'T') -> 'AT': {test_merged}")

Testing BPE functions...
Test tokens: ['A', 'T', 'C', 'G', 'A', 'T']
Pair counts: Counter({('A', 'T'): 2, ('T', 'C'): 1, ('C', 'G'): 1, ('G', 'A'): 1})
After merging ('A', 'T') -> 'AT': ['AT', 'C', 'G', 'AT']


In [9]:
def train_bpe(corpus, target_vocab_size):
    """
    Train BPE tokenizer on corpus

    Args:
        corpus: DNA sequence string
        target_vocab_size: desired vocabulary size

    Returns:
        (vocab, merge_rules, final_tokens)
    """
    print("Starting BPE training...")
    print(f"Corpus size: {len(corpus):,} bases")
    print(f"Target vocab size: {target_vocab_size:,}")
    start_time = time.time()

    # STEP 1: Convert corpus to list of characters (initial tokens)
    # Each character becomes its own token
    tokens = list(corpus)
    print(f"Initial tokens: {len(tokens):,}")

    # STEP 2: Initialize vocabulary with unique characters
    vocab = initialize_vocab(corpus)
    initial_vocab_size = len(vocab)
    print(f"Initial vocab size: {initial_vocab_size}")

    # STEP 3: Calculate how many merges we need
    num_merges = target_vocab_size - initial_vocab_size
    print(f"Number of merges needed: {num_merges:,}")

    # STEP 4: Store merge rules (order matters for encoding later!)
    merge_rules = []

    # STEP 5: Main training loop - perform merges until we hit target vocab size
    print("\nTraining progress:")
    for i in range(num_merges):
        # Count all adjacent pairs in current token list
        pair_counts = get_pair_counts(tokens)

        # Check if we have any pairs left (shouldn't happen, but safety check)
        if not pair_counts:
            print(f"\nNo more pairs to merge at iteration {i}")
            break

        # Find the most frequent pair (greedy approach)
        best_pair = max(pair_counts, key=pair_counts.get)
        best_count = pair_counts[best_pair]

        # Create new token by concatenating the pair
        # e.g., ('A', 'T') becomes 'AT'
        new_token = best_pair[0] + best_pair[1]

        # Add new token to vocabulary with next available ID
        vocab[new_token] = len(vocab)

        # Record this merge rule (needed for encoding later)
        merge_rules.append(best_pair)

        # Actually perform the merge - replace all occurrences in corpus
        tokens = merge_pair(tokens, best_pair, new_token)

        # Print progress every 100 iterations
        if (i + 1) % 100 == 0:
            compression = len(corpus) / len(tokens)
            print(f"  Merge {i+1:>4}/{num_merges}: "
                  f"Merged {best_pair} ({best_count:,} occurrences) -> '{new_token}' | "
                  f"Tokens: {len(tokens):,} | "
                  f"Compression: {compression:.2f}x")

    elapsed = time.time() - start_time
    print(f"\n✓ Training complete in {elapsed:.2f} seconds")
    print(f"Final vocab size: {len(vocab):,}")
    print(f"Final token count: {len(tokens):,}")

    return vocab, merge_rules, tokens


# Train the tokenizer
vocab, merge_rules, compressed_tokens = train_bpe(corpus, TARGET_VOCAB_SIZE)

Starting BPE training...
Corpus size: 4,641,652 bases
Target vocab size: 5,000
Initial tokens: 4,641,652
Initial vocab size: 4
Number of merges needed: 4,996

Training progress:
  Merge  100/4996: Merged ('TT', 'TTC') (2,830 occurrences) -> 'TTTTC' | Tokens: 1,575,822 | Compression: 2.95x
  Merge  200/4996: Merged ('GCC', 'AGC') (1,297 occurrences) -> 'GCCAGC' | Tokens: 1,387,661 | Compression: 3.34x
  Merge  300/4996: Merged ('AT', 'GCCG') (722 occurrences) -> 'ATGCCG' | Tokens: 1,291,770 | Compression: 3.59x
  Merge  400/4996: Merged ('T', 'AGAA') (483 occurrences) -> 'TAGAA' | Tokens: 1,232,932 | Compression: 3.76x
  Merge  500/4996: Merged ('TAAC', 'CC') (372 occurrences) -> 'TAACCC' | Tokens: 1,190,830 | Compression: 3.90x
  Merge  600/4996: Merged ('GC', 'TCG') (291 occurrences) -> 'GCTCG' | Tokens: 1,158,195 | Compression: 4.01x
  Merge  700/4996: Merged ('T', 'ACCCG') (241 occurrences) -> 'TACCCG' | Tokens: 1,131,648 | Compression: 4.10x
  Merge  800/4996: Merged ('TCTG', 'GG')

In [10]:
def calculate_compression_ratio(original_corpus, compressed_tokens):
    """
    Calculate compression ratio

    Args:
        original_corpus: original DNA string
        compressed_tokens: list of tokens after BPE

    Returns:
        float: compression ratio
    """
    # Original size = number of bytes (characters in string)
    original_size = len(original_corpus)

    # Compressed size = number of tokens after merging
    compressed_size = len(compressed_tokens)

    # Compression ratio = how many bytes each token represents on average
    compression_ratio = original_size / compressed_size

    return compression_ratio


# Calculate metrics
compression_ratio = calculate_compression_ratio(corpus, compressed_tokens)

print("\n" + "="*60)
print("FINAL RESULTS")
print("="*60)
print(f"Vocabulary size: {len(vocab):,} tokens")
print(f"Original size: {len(corpus):,} bytes (characters)")
print(f"Compressed size: {len(compressed_tokens):,} tokens")
print(f"Compression ratio: {compression_ratio:.3f}x")
print(f"Average token length: {len(corpus) / len(compressed_tokens):.3f} bases")

# Additional statistics
avg_token_length = sum(len(token) for token in vocab.keys()) / len(vocab)
max_token_length = max(len(token) for token in vocab.keys())
print(f"Average token length (vocab): {avg_token_length:.2f} bases")
print(f"Longest token: {max_token_length} bases")

# Check requirements
print("\n" + "="*60)
print("REQUIREMENTS CHECK")
print("="*60)
vocab_check = "✓ PASS" if len(vocab) >= 5000 else "✗ FAIL"
compression_check = "✓ PASS" if compression_ratio >= 3.2 else "✗ FAIL"

print(f"{vocab_check} Vocabulary size ≥ 5000: {len(vocab):,}")
print(f"{compression_check} Compression ratio ≥ 3.2: {compression_ratio:.3f}")

if len(vocab) >= 5000 and compression_ratio >= 3.2:
    print("\n🎉 ALL REQUIREMENTS MET! 🎉")
else:
    print("\n⚠️  Some requirements not met. Consider training longer.")


FINAL RESULTS
Vocabulary size: 5,000 tokens
Original size: 4,641,652 bytes (characters)
Compressed size: 891,316 tokens
Compression ratio: 5.208x
Average token length: 5.208 bases
Average token length (vocab): 6.91 bases
Longest token: 26 bases

REQUIREMENTS CHECK
✓ PASS Vocabulary size ≥ 5000: 5,000
✓ PASS Compression ratio ≥ 3.2: 5.208

🎉 ALL REQUIREMENTS MET! 🎉


In [11]:
def analyze_vocabulary(vocab, merge_rules):
    """
    Analyze what patterns BPE learned

    Args:
        vocab: trained vocabulary dict
        merge_rules: list of merge operations
    """
    print("\n" + "="*60)
    print("VOCABULARY ANALYSIS")
    print("="*60)

    # SECTION 1: Token length distribution
    print("\n1. TOKEN LENGTH DISTRIBUTION")
    print("-" * 60)

    token_lengths = {}
    for token in vocab.keys():
        length = len(token)
        token_lengths[length] = token_lengths.get(length, 0) + 1

    # Sort by length and display
    for length in sorted(token_lengths.keys()):
        count = token_lengths[length]
        bar = "█" * (count // 50)  # Visual bar chart
        print(f"  Length {length:2d}: {count:4d} tokens {bar}")


    # SECTION 2: Biologically meaningful patterns
    print("\n2. BIOLOGICAL PATTERNS FOUND")
    print("-" * 60)

    # Define known biological motifs
    biological_patterns = {
        'Start Codons': ['ATG'],
        'Stop Codons': ['TAA', 'TAG', 'TGA'],
        'TATA Box': ['TATA', 'TATAA', 'TATAAA', 'TATATAT'],
        'Shine-Dalgarno (ribosome binding)': ['AGGAGG', 'AGGA', 'GGAGG'],
        'Poly-A tail': ['AAA', 'AAAA', 'AAAAA', 'AAAAAA'],
        'Poly-T': ['TTT', 'TTTT', 'TTTTT'],
        'CpG dinucleotides': ['CG', 'CGCG', 'GCGC'],
        'Homopolymer runs': ['GGG', 'GGGG', 'CCC', 'CCCC'],
    }

    found_any = False
    for category, patterns in biological_patterns.items():
        found_patterns = [p for p in patterns if p in vocab]
        if found_patterns:
            found_any = True
            print(f"\n  {category}:")
            for pattern in found_patterns:
                print(f"    ✓ {pattern} (ID: {vocab[pattern]})")

    if not found_any:
        print("  No well-known biological patterns found in vocabulary")
        print("  (This is normal - BPE learns statistical patterns, not biology)")


    # SECTION 3: Most common initial merges
    print("\n3. FIRST 20 MERGES (Most Common Patterns)")
    print("-" * 60)

    for i, pair in enumerate(merge_rules[:20]):
        merged_token = pair[0] + pair[1]
        print(f"  Merge {i+1:2d}: {pair[0]:8s} + {pair[1]:8s} = {merged_token}")


    # SECTION 4: Longest tokens learned
    print("\n4. LONGEST TOKENS (Top 20)")
    print("-" * 60)

    # Sort all tokens by length
    sorted_by_length = sorted(vocab.keys(), key=len, reverse=True)

    for i, token in enumerate(sorted_by_length[:20]):
        print(f"  {i+1:2d}. {token:20s} (length: {len(token)}, ID: {vocab[token]})")


    # SECTION 5: Sample random tokens from middle of vocabulary
    print("\n5. SAMPLE TOKENS FROM MIDDLE OF TRAINING")
    print("-" * 60)

    # Get tokens created around merge 2500 (middle of training)
    if len(merge_rules) >= 2500:
        middle_merges = merge_rules[2400:2420]
        print("  Tokens created around merge #2500:")
        for pair in middle_merges:
            merged = pair[0] + pair[1]
            print(f"    {merged}")


    # SECTION 6: Character frequency in vocabulary
    print("\n6. CHARACTER FREQUENCY IN VOCABULARY TOKENS")
    print("-" * 60)

    char_counts = Counter()
    for token in vocab.keys():
        char_counts.update(token)

    total_chars = sum(char_counts.values())
    for char in sorted(char_counts.keys()):
        count = char_counts[char]
        percentage = (count / total_chars) * 100
        bar = "█" * int(percentage)
        print(f"  {char}: {count:6d} ({percentage:5.2f}%) {bar}")


# Run analysis
analyze_vocabulary(vocab, merge_rules)


VOCABULARY ANALYSIS

1. TOKEN LENGTH DISTRIBUTION
------------------------------------------------------------
  Length  1:    4 tokens 
  Length  2:   10 tokens 
  Length  3:   25 tokens 
  Length  4:   89 tokens █
  Length  5:  300 tokens ██████
  Length  6:  971 tokens ███████████████████
  Length  7: 2284 tokens █████████████████████████████████████████████
  Length  8: 1184 tokens ███████████████████████
  Length  9:  109 tokens ██
  Length 10:    9 tokens 
  Length 11:    4 tokens 
  Length 12:    2 tokens 
  Length 13:    3 tokens 
  Length 16:    3 tokens 
  Length 18:    2 tokens 
  Length 26:    1 tokens 

2. BIOLOGICAL PATTERNS FOUND
------------------------------------------------------------

  Start Codons:
    ✓ ATG (ID: 20)

  Stop Codons:
    ✓ TAA (ID: 25)
    ✓ TAG (ID: 65)

  TATA Box:
    ✓ TATAA (ID: 279)

  Shine-Dalgarno (ribosome binding):
    ✓ AGGAGG (ID: 3642)

  Poly-A tail:
    ✓ AAAA (ID: 34)
    ✓ AAAAAA (ID: 219)

  Poly-T:
    ✓ TTT (ID: 55)
    ✓ TTT

In [13]:
def encode(text, vocab, merge_rules):
    """
    Encode new text using trained tokenizer

    Args:
        text: DNA sequence string to encode
        vocab: trained vocabulary
        merge_rules: list of merge rules from training

    Returns:
        list of token IDs
    """
    # STEP 1: Start with character-level tokenization
    tokens = list(text)

    # STEP 2: Apply all merge rules in the SAME ORDER as training
    # This is critical! Order matters in BPE
    for pair in merge_rules:
        # Create the merged token string
        merged_token = pair[0] + pair[1]

        # Replace all occurrences of this pair
        tokens = merge_pair(tokens, pair, merged_token)

    # STEP 3: Convert tokens to IDs using vocabulary
    token_ids = []
    for token in tokens:
        if token in vocab:
            token_ids.append(vocab[token])
        else:
            # This shouldn't happen if encoding same distribution as training
            print(f"Warning: Unknown token '{token}' - skipping")

    return token_ids


def decode(token_ids, vocab):
    """
    Decode token IDs back to original text

    Args:
        token_ids: list of integer token IDs
        vocab: trained vocabulary

    Returns:
        string: decoded DNA sequence
    """
    # STEP 1: Create reverse vocabulary (ID -> token string)
    # This maps numbers back to the actual token strings
    id_to_token = {idx: token for token, idx in vocab.items()}

    # STEP 2: Look up each ID and get its token string
    tokens = []
    for token_id in token_ids:
        if token_id in id_to_token:
            tokens.append(id_to_token[token_id])
        else:
            print(f"Warning: Unknown token ID {token_id}")

    # STEP 3: Concatenate all token strings to reconstruct original
    decoded_text = ''.join(tokens)

    return decoded_text


# ============================================================
# TEST ENCODE/DECODE
# ============================================================

print("\n" + "="*60)
print("ENCODE/DECODE TESTING")
print("="*60)

# Test 1: Small sequence
print("\nTest 1: Small sequence")
print("-" * 60)
test_seq = "ATCGATCGATCGATCG"
print(f"Original: {test_seq}")

encoded = encode(test_seq, vocab, merge_rules)
print(f"Encoded:  {encoded}")
print(f"Length:   {len(test_seq)} chars -> {len(encoded)} tokens")
print(f"Compression: {len(test_seq) / len(encoded):.2f}x")

decoded = decode(encoded, vocab)
print(f"Decoded:  {decoded}")
print(f"Lossless: {test_seq == decoded} ✓" if test_seq == decoded else f"Lossless: {test_seq == decoded} ✗")


# Test 2: Longer sequence from corpus
print("\nTest 2: Real genome sequence (1000 bases)")
print("-" * 60)
test_sequence = corpus[:1000]

encoded = encode(test_sequence, vocab, merge_rules)
decoded = decode(encoded, vocab)

print(f"Original length: {len(test_sequence):,} bases")
print(f"Encoded length:  {len(encoded):,} tokens")
print(f"Compression:     {len(test_sequence)/len(encoded):.3f}x")
print(f"Lossless check:  {test_sequence == decoded} ✓" if test_sequence == decoded else f"Lossless: {test_sequence == decoded} ✗")
print(f"\nFirst 10 tokens: {encoded[:10]}")
print(f"First 10 decoded strings: {[decode([tid], vocab) for tid in encoded[:10]]}")


# Test 3: Entire corpus (verify overall compression)
print("\nTest 3: Entire E. coli genome")
print("-" * 60)
full_encoded = encode(corpus, vocab, merge_rules)
print(f"Full genome tokens: {len(full_encoded):,}")
print(f"Compression ratio:  {len(corpus) / len(full_encoded):.3f}x")

# Verify it matches our training compression
print(f"Training tokens:    {len(compressed_tokens):,}")
print(f"Match:              {len(full_encoded) == len(compressed_tokens)} ✓" if len(full_encoded) == len(compressed_tokens) else f"Match: {len(full_encoded) == len(compressed_tokens)} ⚠️")


# Test 4: Show some example encodings
print("\nTest 4: Example token encodings")
print("-" * 60)

examples = [
    "ATG",      # Start codon
    "TAA",      # Stop codon
    "AAAA",     # Poly-A
    "GCGCGC",   # Repeating pattern
]

for seq in examples:
    if seq in corpus[:100000]:  # Check if it exists in genome
        enc = encode(seq, vocab, merge_rules)
        dec = decode(enc, vocab)
        tokens_str = [decode([tid], vocab) for tid in enc]
        print(f"{seq:12s} -> {str(enc):20s} -> {tokens_str} -> {dec}")


ENCODE/DECODE TESTING

Test 1: Small sequence
------------------------------------------------------------
Original: ATCGATCGATCGATCG
Encoded:  [44, 44, 44, 44]
Length:   16 chars -> 4 tokens
Compression: 4.00x
Decoded:  ATCGATCGATCGATCG
Lossless: True ✓

Test 2: Real genome sequence (1000 bases)
------------------------------------------------------------
Original length: 1,000 bases
Encoded length:  198 tokens
Compression:     5.051x
Lossless check:  True ✓

First 10 tokens: [21, 103, 43, 143, 22, 160, 3697, 1265, 9, 555]
First 10 decoded strings: ['AGC', 'TTTTC', 'ATTC', 'TGAC', 'TGC', 'AACGG', 'GCAATATG', 'TCTCTG', 'TG', 'TGGATT']

Test 3: Entire E. coli genome
------------------------------------------------------------
Full genome tokens: 891,316
Compression ratio:  5.208x
Training tokens:    891,316
Match:              True ✓

Test 4: Example token encodings
------------------------------------------------------------
ATG          -> [20]                 -> ['ATG'] -> ATG
TAA  